In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import pinv
from scipy.linalg import null_space
#data
np.random.seed(42)
n = 2 # No of State Rows X - A
m = 1 #No of i/p cls U - B
sample = 5

# A and B Matrices for Mode 1 with integer values between 0 and 10
A1 = np.random.randint(0, 10, size=(n, n))
B1 = np.random.randint(0, 10, size=(n, m))

# A and B Matrices for Mode 2 with integer values between 0 and 10
A2 = np.random.randint(0, 10, size=(n, n))
B2 = np.random.randint(0, 10, size=(n, m))
print("A1:", A1)
print("B1:", B1)
print("A2:", A2)
print("B2:", B2)

A1: [[6 3]
 [7 4]]
B1: [[6]
 [9]]
A2: [[2 6]
 [7 4]]
B2: [[3]
 [7]]


In [3]:
# Generate random integer data for X1 and U1 (values between -10 and 10)
X1_ = np.random.randint(-10, 11, size=(n, sample))
U1_ = np.random.randint(-10, 11, size=(m, sample))

# Generate random integer data for X2 and U2 (values between -10 and 10)
X2_ = np.random.randint(-10, 11, size=(n, sample))
U2_ = np.random.randint(-10, 11, size=(m, sample))

print("X1_:\n", X1_)
print("U1_:\n", U1_)
print("X2_:\n", X2_)
print("U2_:\n", U2_)

X1_:
 [[ -8  10  -9   1  -5]
 [ -9  10 -10   1   1]]
U1_:
 [[ 6 -1  5  4  4]]
X2_:
 [[ 8  1  9 -8 -6]
 [ 8 -4 10 -2 -4]]
U2_:
 [[ 7 -7  3  7 -2]]


In [5]:
# Simulate X_plus for both modes (noiseless)
X1_plus = A1 @ X1_ + B1 @ U1_
X2_plus = A2 @ X2_ + B2 @ U2_
print (X1_plus)
print (X2_plus)

[[-39  84 -54  33  -3]
 [-38 101 -58  47   5]]
[[ 85 -43  87  -7 -42]
 [137 -58 124 -15 -72]]


In [7]:
# Concatenate X_minus and U_minus
Z1 = np.vstack((X1_, U1_))
Z2 = np.vstack((X2_, U2_))

# Θ=[AB] and z=concate matrix
Theta1_est = X1_plus @ pinv(Z1)
Theta2_est = X2_plus @ pinv(Z2)

# Extract A and B from Theta
A1_est = Theta1_est[:, :n]
B1_est = Theta1_est[:, n:]

A2_est = Theta2_est[:, :n]
B2_est = Theta2_est[:, n:]

print("Estimated A1:\n", A1_est)
print("Estimated B1:\n", B1_est)
print("Estimated A2:\n", A2_est)
print("Estimated B2:\n", B2_est)


Estimated A1:
 [[6. 3.]
 [7. 4.]]
Estimated B1:
 [[6.]
 [9.]]
Estimated A2:
 [[2. 6.]
 [7. 4.]]
Estimated B2:
 [[3.]
 [7.]]


In [32]:
# Set random seed for reproducibility
np.random.seed(42)

# System dimensions
n = 2  # Number of states
m = 1  # Number of inputs
T_on = 10  # Time horizon for online data generation

# Initial state for online data generation
X_on = np.random.randint(-10, 11, size=(n, 1))

# Containers to store online data
X_on_data = []
U_on_data = []

# Generate online inputs (random for now)
for t in range(T_on):
    # Random input between -10 and 10
    U_on = np.random.randint(-10, 11, size=(m, 1))
    
    # Simulate next state for Mode 1 (you can choose Mode 2 by replacing A1_est, B1_est)
    X_on_next = A1_est @ X_on + B1_est @ U_on  # No noise added

    # Store the data
    X_on_data.append(X_on_next)
    U_on_data.append(U_on)

    # Update the current state
    X_on = X_on_next

# Convert lists to numpy arrays for easier manipulation
X_on_data = np.hstack(X_on_data)  # Shape: (n, T_on)
U_on_data = np.hstack(U_on_data)  # Shape: (m, T_on)

# Display generated online data
print("Generated Online State Data (X_on):\n", X_on_data)
print("Generated Online Input Data (U_on):\n", U_on_data)


Generated Online State Data (X_on):
 [[         27         294        2841       27579      267273     2590029
     25098495   243214863 -1938114091  1364050969]
 [         44         365        3491       33941      328781     3186107
     30874631   299187989 -1395711209 -1969774352]]
Generated Online Input Data (U_on):
 [[ 4  0 -3 10 -4  8  0  0 10 -7]]


In [43]:
# Function to check controllability using controllability matrix
def is_controllable(A, B):
    n = A.shape[0]
    controllability_matrix = B
    for i in range(1, n):
        controllability_matrix = np.hstack((controllability_matrix, np.linalg.matrix_power(A, i) @ B))
    rank = matrix_rank(controllability_matrix)
    return controllability_matrix, rank == n, rank

# Check controllability for Mode 1 and Mode 2
controllability_matrix_mode1, controllable_mode1, rank_mode1 = is_controllable(A1_est, B1_est)
controllability_matrix_mode2, controllable_mode2, rank_mode2 = is_controllable(A2_est, B2_est)

print(f"Mode 1 is controllable: {controllable_mode1}")
print(f"Controllability Matrix for Mode 1:\n{controllability_matrix_mode1}")
print(f"Rank of Controllability Matrix for Mode 1: {rank_mode1}\n")

print(f"Mode 2 is controllable: {controllable_mode2}")
print(f"Controllability Matrix for Mode 2:\n{controllability_matrix_mode2}")
print(f"Rank of Controllability Matrix for Mode 2: {rank_mode2}\n")

Mode 1 is controllable: True
Controllability Matrix for Mode 1:
[[ 6 63]
 [ 9 78]]
Rank of Controllability Matrix for Mode 1: 2

Mode 2 is controllable: True
Controllability Matrix for Mode 2:
[[ 3 48]
 [ 7 49]]
Rank of Controllability Matrix for Mode 2: 2



In [55]:
# Check rank condition: rank([X_on^- ; U_on^-]) == n + m
# Using first (n + m) columns for the condition
X_on_minus = X_on_data[:, :n + m]
U_on_minus = U_on_data[:, :n + m]

# Concatenate vertically
combined_matrix = np.vstack((X_on_minus, U_on_minus))

# Compute the rank
rank_combined = matrix_rank(combined_matrix)

print('Combined data Matrix Uon and Xon: \n', combined_matrix)

print(f"Rank of the combined matrix [X_on^- ; U_on^-]: {rank_combined}")
print(f"Expected rank (n + m): {n + m}")

if rank_combined == n + m:
    print("Rank condition satisfied: The set Σ(U_on^-, X_on^-) is a singleton.")
else:
    print("Rank condition NOT satisfied: The set Σ(U_on^-, X_on^-) is not a singleton.")


Combined data Matrix Uon and Xon: 
 [[  27  294 2841]
 [  44  365 3491]
 [   4    0   -3]]
Rank of the combined matrix [X_on^- ; U_on^-]: 3
Expected rank (n + m): 3
Rank condition satisfied: The set Σ(U_on^-, X_on^-) is a singleton.


In [67]:
#Non Singleton Scenario

# Initial state for online data generation
X_on = np.array([[1], [1]])  # Fixed initial state to introduce dependency

# Containers to store online data
X_on_data = []
U_on_data = []

# Generate online inputs designed to create rank deficiency
for t in range(T_on):
    # Linearly dependent inputs (e.g., multiples of a base input)
    U_on = np.array([[2 * t]])  # Linearly increasing input to ensure dependency
    
    # Simulate next state for Mode 1 (you can choose Mode 2 by replacing A1_est, B1_est)
    X_on_next = A1_est @ X_on + B1_est @ U_on  # No noise added

    # Force linear dependency in states by making each new state a linear combination of the first
    if t > 0:
        X_on_next = 2 * X_on_data[0]

    # Store the data
    X_on_data.append(X_on_next)
    U_on_data.append(U_on)

    # Update the current state
    X_on = X_on_next

# Convert lists to numpy arrays for easier manipulation
X_on_data = np.hstack(X_on_data)  # Shape: (n, T_on)
U_on_data = np.hstack(U_on_data)  # Shape: (m, T_on)

# Display generated online data
print("Generated Online State Data (X_on):\n", X_on_data)
print("Generated Online Input Data (U_on):\n", U_on_data)

# Check rank condition: rank([X_on^- ; U_on^-]) == n + m
# Using first (n + m) columns for the condition
X_on_minus = X_on_data[:, :n + m]
U_on_minus = U_on_data[:, :n + m]

# Concatenate vertically
combined_matrix = np.vstack((X_on_minus, U_on_minus))
print(combined_matrix)
# Compute the rank
rank_combined = matrix_rank(combined_matrix)

print(f"Rank of the combined matrix [X_on^- ; U_on^-]: {rank_combined}")
print(f"Expected rank (n + m): {n + m}")

if rank_combined == n + m:
    print("Rank condition satisfied: The set Σ(U_on^-, X_on^-) is a singleton.")
else:
    print("Rank condition NOT satisfied: The set Σ(U_on^-, X_on^-) is not a singleton.")

Generated Online State Data (X_on):
 [[ 9 18 18 18 18 18 18 18 18 18]
 [11 22 22 22 22 22 22 22 22 22]]
Generated Online Input Data (U_on):
 [[ 0  2  4  6  8 10 12 14 16 18]]
[[ 9 18 18]
 [11 22 22]
 [ 0  2  4]]
Rank of the combined matrix [X_on^- ; U_on^-]: 2
Expected rank (n + m): 3
Rank condition NOT satisfied: The set Σ(U_on^-, X_on^-) is not a singleton.


In [81]:
# Active State Check for u(t) selection
from numpy.linalg import matrix_rank, lstsq
# Check if x(t) is in the image of X_on^-
def is_in_image(X_on_minus, x_t):
    solution, residuals, rank, s = lstsq(X_on_minus, x_t, rcond=None)
    print(solution)
    print(residuals)
    print(rank)
    print(s)
    return residuals.size == 0 or np.allclose(residuals, 0)

# Example current state x(t)
x_t = np.array([[5], [5]])  # Replace with actual state as needed

if is_in_image(X_on_minus, x_t):
    print(f"x(t) is in the image of X_on^-. Proceeding to input selection.")
else:
    u_t = np.zeros((m, 1))
    print(f"x(t) is NOT in the image of X_on^-. Setting u(t) = {u_t.flatten()}")

[[0.0550055]
 [0.110011 ]
 [0.110011 ]]
[]
1
[42.63801121  0.        ]
x(t) is in the image of X_on^-. Proceeding to input selection.


In [83]:
from numpy.linalg import matrix_rank, pinv

# Function to check if x(t) is in the image of X_on^-
def is_in_image(X_on_minus, x_t):
    rank_original = matrix_rank(X_on_minus)
    combined_matrix = np.hstack((X_on_minus, x_t.reshape(-1, 1)))
    rank_combined = matrix_rank(combined_matrix)
    return rank_original == rank_combined

# Function to find eta and xi if x(t) is in the image of X_on^-
def find_eta_xi(X_on_minus, U_on_minus):
    combined_matrix = np.vstack((X_on_minus, U_on_minus))
    null_space_vector = null_space(combined_matrix.T)
    if null_space_vector.size > 0:
        xi = null_space_vector[:X_on_minus.shape[0], 0]
        eta = null_space_vector[X_on_minus.shape[0]:, 0]
        return xi, eta
    return None, None

# Function to choose u(t) such that xi^T x(t) + eta^T u(t) = 0
def choose_u_t(xi, eta, x_t, c=1):
    if eta is not None and np.linalg.norm(eta) != 0:
        u_t = - (xi.T @ x_t) / (eta.T @ eta) * eta
        if np.linalg.norm(u_t) > c * np.linalg.norm(x_t):
            u_t = (c * np.linalg.norm(x_t) / np.linalg.norm(u_t)) * u_t
        return u_t
    return np.zeros((U_on_minus.shape[0], 1))


In [91]:
# Example usage
x_t = np.random.randint(-10, 11, size=(X_on_data.shape[0], 1))
if not is_in_image(X_on_data, x_t):
    u_t = np.zeros((U_on_data.shape[0], 1))
else:
    xi, eta = find_eta_xi(X_on_data, U_on_data)
    u_t = choose_u_t(xi, eta, x_t)

# Append new data to X_on_data and U_on_data
X_on_data = np.hstack((X_on_data, x_t))
U_on_data = np.hstack((U_on_data, u_t))

# Display updated data
print("Updated Online State Data (X_on):\n", X_on_data)
print("Updated Online Input Data (U_on):\n", U_on_data) 

# Check rank condition
combined_matrix = np.vstack((X_on_data, U_on_data))
print('Combined data: \n',combined_matrix)
rank_combined = matrix_rank(combined_matrix)
print(f"Rank of combined matrix: {rank_combined}")


Updated Online State Data (X_on):
 [[  9  18  18  18  18  18  18  18  18  18  -9 -10   1]
 [ 11  22  22  22  22  22  22  22  22  22  10   1   6]]
Updated Online Input Data (U_on):
 [[ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.  0.  0.  0.]]
Combined data: 
 [[  9.  18.  18.  18.  18.  18.  18.  18.  18.  18.  -9. -10.   1.]
 [ 11.  22.  22.  22.  22.  22.  22.  22.  22.  22.  10.   1.   6.]
 [  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.   0.   0.   0.]]
Rank of combined matrix: 3
